### Operator overloading for more concise model definitions

Thinc allows you to **overload operators** and bind arbitrary functions to Python operators like `+`, `*`, but also `>>` or `@`. The `Model.define_operators` contextmanager takes a dict of operators mapped to functions – typically combinators like `chain`. The operators are only valid for the `with` block. This lets us define the model like this:

In [4]:
from thinc.api import prefer_gpu
prefer_gpu()

False

Instead of defining the `chain` as a comma-separated list of elements, one can use custom operators.
For example, transforming the following code 

```python
from thinc.api import Model, chain, Relu, Softmax
n_hidden = 32
dropout = 0.2

model = chain(
    Relu(nO=n_hidden, dropout=dropout), 
    Relu(nO=n_hidden, dropout=dropout), 
    Softmax()
)
```


into this:

In [8]:
from thinc.api import Model, chain, Relu, Softmax

 
n_hidden = 32
dropout = 0.2

with Model.define_operators({">>": chain}):
    model = Relu(nO=n_hidden, dropout=dropout) >> Relu(nO=n_hidden, dropout=dropout) >> Softmax()

You can now use the `model` object as an argument to the `train_model` function defined below

In [9]:
def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_Y), (dev_X, dev_Y) = data
    indices = model.ops.xp.arange(train_X.shape[0], dtype="i")
    for i in range(n_iter):
        batches = model.ops.multibatch(batch_size, train_X, train_Y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            backprop(Yh - Y)
            model.finish_update(optimizer)
        # Evaluate and print progress
        correct = 0
        total = 0
        for X, Y in model.ops.multibatch(batch_size, dev_X, dev_Y):
            Yh = model.predict(X)
            correct += (Yh.argmax(axis=1) == Y.argmax(axis=1)).sum()
            total += Yh.shape[0]
        score = correct / total
        print(f" {i} {float(score):.3f}")

In [11]:
from thinc.api import Adam, fix_random_seed
from tqdm.notebook import tqdm
import ml_datasets


fix_random_seed(0)
optimizer = Adam(0.001)
batch_size = 128
data = (train_X, train_Y), (dev_X, dev_Y) = ml_datasets.mnist()


print("Measuring performance across iterations:")
train_model(data, model, optimizer, 20, batch_size)



Measuring performance across iterations:


KeyError: "Parameter 'W' for model 'relu' has not been allocated yet."